# <font color=red> Malaria_Cell_Images

<img src='malaria.jpg'>

In [3]:
import os
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pathlib
import PIL.Image as Image

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense


import warnings
warnings.filterwarnings('ignore')

In [4]:
#pip install opencv_python

In [5]:
labels=['Parasitized','Uninfected'] #labellerimizi tanimladik
img_path='cell_images/' # yukaridaki tanimladigimiz labelleri hangi dosyadan cekicegimizi tanimladik

In [6]:
img_list=[] # okudugumuz resimlerin isimlerini kaydetmek icin liste olusturduk
label_list=[] # okudugumuz resimlerin Parasitized yada Uninfected olup olmadigini yanina yazmak icin liste olurturduk.
for label in labels: # labels listesinin icindeki labelleri okuduk
    for img_file in os.listdir(img_path+label): #img_file diye bir data dosyasini olusturup icinde img_path+label listdir kodu ile tüm dosya islemlerini listeledik
        img_list.append(img_path+label+'/'+img_file) # tüm dosyalarin tam yollarini bir listeye toplayip en sona .append ile yeni bir eleman ekledik
        label_list.append(label) # .append listeye yeni bir eleman eklemeye yariyor.

In [7]:
df=pd.DataFrame({'img':img_list, 'label':label_list})

In [8]:
df.head()

,img,label
0,cell_images/Parasitized/C100P61ThinF_IMG_20150...,Parasitized
1,cell_images/Parasitized/C100P61ThinF_IMG_20150...,Parasitized
2,cell_images/Parasitized/C100P61ThinF_IMG_20150...,Parasitized
3,cell_images/Parasitized/C100P61ThinF_IMG_20150...,Parasitized
4,cell_images/Parasitized/C100P61ThinF_IMG_20150...,Parasitized


In [9]:
df.tail()

,img,label
27555,cell_images/Uninfected/C99P60ThinF_IMG_2015091...,Uninfected
27556,cell_images/Uninfected/C99P60ThinF_IMG_2015091...,Uninfected
27557,cell_images/Uninfected/C99P60ThinF_IMG_2015091...,Uninfected
27558,cell_images/Uninfected/C99P60ThinF_IMG_2015091...,Uninfected
27559,cell_images/Uninfected/Thumbs.db,Uninfected


In [10]:
d={'Parasitized':1,'Uninfected':0}

In [11]:
df['encode_label']=df['label'].map(d)

In [12]:
df.head()

,img,label,encode_label
0,cell_images/Parasitized/C100P61ThinF_IMG_20150...,Parasitized,1
1,cell_images/Parasitized/C100P61ThinF_IMG_20150...,Parasitized,1
2,cell_images/Parasitized/C100P61ThinF_IMG_20150...,Parasitized,1
3,cell_images/Parasitized/C100P61ThinF_IMG_20150...,Parasitized,1
4,cell_images/Parasitized/C100P61ThinF_IMG_20150...,Parasitized,1


In [13]:
df.tail()

,img,label,encode_label
27555,cell_images/Uninfected/C99P60ThinF_IMG_2015091...,Uninfected,0
27556,cell_images/Uninfected/C99P60ThinF_IMG_2015091...,Uninfected,0
27557,cell_images/Uninfected/C99P60ThinF_IMG_2015091...,Uninfected,0
27558,cell_images/Uninfected/C99P60ThinF_IMG_2015091...,Uninfected,0
27559,cell_images/Uninfected/Thumbs.db,Uninfected,0


In [38]:
x=[]
y=[]
for img_path in df['img']:
    if not img_path.endswith(('.jpg', '.jpeg', '.png')):
        continue
    img = cv2.imread(img_path)
    if img is None:
        print(f"Resim okunamadı: {img_path}")
        continue
    img = cv2.resize(img, (30, 30))
    img = img / 255.0
    x.append(img)
    y.append(label)

In [39]:
x = np.array(x)
y = np.array(y)

# <font color=red> CNN

In [42]:
x_train,x_test, y_train,y_test=train_test_split(x,y,random_state=42, test_size=0.20)

In [46]:
from tensorflow.keras.layers import Input

In [48]:
model = Sequential()
model.add(Input(shape=(30,30,3)))
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(250))
model.add(Dense(150))
model.add(Dense(100))
model.add(Dense(2, activation='softmax'))  # Added activation function

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [56]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=15, verbose=1)

Epoch 1/15
689/689 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.9965 - loss: 0.0071 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/15
689/689 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/15
689/689 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/15
689/689 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/15
689/689 ━━━━━━━━━━━━━━━━━━━━ 8s 12ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 6/15
689/689 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 7/15
689/689 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 8/15
689/689 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/s

In [61]:
model.save('my_cnn_model.h5')